In [4]:
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# Input and output paths
# --------------------------
image_path = r"C:\Users\DELL\Downloads\fullImages\fullImages\set1_Image_01_40x_bf_02.png"
output_folder = r"C:\Users\DELL\Downloads\40x_bf_02_qwerty5_4_statistics_regrowth_all"
os.makedirs(output_folder, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245
MIN_CIRCULARITY = 0.10
MIN_CONTOUR_AREA = 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.3  # adjust based on dataset

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)
    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def mean_intensity_in_contour(gray, contour):
    m = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(m, [contour], -1, 255, -1)
    return cv2.mean(gray, mask=m)[0]

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset, axon_data, object_counter, output_image):
    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    # Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)

    # Use both Otsu + adaptive threshold (union of both)
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )
    thresh = cv2.bitwise_or(otsu, adaptive)

    # Clean
    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    contours, hierarchy = cv2.findContours(cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):
        # only outer contours
        if hierarchy[i][3] != -1:
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # Collect all inner children
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)

        if len(inner_children) == 0:
            continue

        # Classification
        axon_type = "mature" if len(inner_children) == 1 else "regrowth_cluster"

        # Largest inner for measurement
        largest_inner = max(inner_children, key=cv2.contourArea)

        # --- Outer ---
        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset
        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor**2)

        # Draw outer boundary with type-specific color
        outer_color = (255, 0, 0) if axon_type == "mature" else (0, 255, 255)
        cv2.drawContours(output_image, [contour_shifted], -1, outer_color, 1)

        # --- Inner ---
        # --- Inner ---
        inner_radii = []
        inner_areas = []
        
        for inner_c in inner_children:
            (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(inner_c)
            inner_shifted = inner_c + np.array([x_offset, y_offset])
            cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)  # Draw all in green
            inner_radii.append(inner_radius)
            inner_areas.append(cv2.contourArea(inner_c))
        
        # Use the largest inner contour for metrics
        largest_inner = inner_children[np.argmax(inner_areas)]
        inner_radius = max(inner_radii)
        inner_area_px = max(inner_areas)
        inner_area_um2 = inner_area_px * (scale_factor ** 2)

        # Metrics
        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)
        area_ratio = (inner_area_um2 / outer_area_um2) if outer_area_um2 > 0 else 0
        hole_ratio = inner_area_um2 / (outer_area_um2 + 1e-6)

        # Create masks for mean intensity
        mask_outer = np.zeros(gray.shape, dtype=np.uint8)
        mask_inner = np.zeros(gray.shape, dtype=np.uint8)
        cv2.drawContours(mask_outer, [contour], -1, 255, -1)
        cv2.drawContours(mask_inner, [largest_inner], -1, 255, -1)

        outer_mean = cv2.mean(gray, mask=mask_outer)[0]
        inner_mean = cv2.mean(gray, mask=mask_inner)[0]
        ring_contrast = (outer_mean - inner_mean) / (outer_mean + 1e-6)

        # LAB a-channel mean inside axon center
        lab = cv2.cvtColor(patch, cv2.COLOR_BGR2LAB)
        _, A, _ = cv2.split(lab)
        a_mean_center = cv2.mean(A, mask=mask_inner)[0]

        # Eccentricity
        eccentricity = 0
        if len(contour) >= 5:
            ellipse = cv2.fitEllipse(contour)
            (MA, ma) = ellipse[1]
            if MA > 0:
                eccentricity = np.sqrt(1 - (ma / MA) ** 2)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "num_inner_contours": len(inner_children),
            "center_x": x_outer,
            "center_y": y_outer,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "area_ratio": area_ratio,
            "hole_ratio": hole_ratio,
            "ring_contrast": ring_contrast,
            "a_mean_center": a_mean_center,
            "eccentricity": eccentricity,
            "g_ratio": g_ratio
        })

        # Numbering
        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

        print(f"Axon {object_counter} ({axon_type}):")
        print(f" Center = ({x_outer:.2f}, {y_outer:.2f})")
        print(f" Outer Radius = {outer_radius*scale_factor:.4f} µm")
        print(f" Inner Radius = {inner_radius*scale_factor:.4f} µm")
        print(f" Thickness = {thickness:.4f} µm")
        print(f" Diameter = {(2*outer_radius)*scale_factor:.4f} µm")
        print(f" Outer Area = {outer_area_um2:.4f} µm²")
        print(f" Inner Area = {inner_area_um2:.4f} µm²")
        print(f" G-Ratio = {g_ratio:.4f}")
        print(f" Inner Contours = {len(inner_children)}\n")

        object_counter += 1

    return object_counter

# --------------------------
# Whole image tiling
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError("Could not read image")

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]
    output_image = image.copy()
    axon_data = []
    object_counter = 1

    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]
            object_counter = process_patch(
                patch, mask_patch, x, y, axon_data, object_counter, output_image
            )

    filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, f"numbered_{filename}")
    cv2.imwrite(output_path, output_image)

    print(f"\nProcessed: {filename}")
    print(f"Axons detected: {len(axon_data)}\n")
    return axon_data, output_image

# --------------------------
# Save data to CSV
# --------------------------
def save_to_csv(axon_data, output_folder, filename="axon_measurements.csv"):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, filename)
    df.to_csv(csv_path, index=False)
    print(f"✅ Axon data saved to: {csv_path}")
    return df

# --------------------------
# Global Histograms
# --------------------------
def plot_global_histograms(df, output_folder):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        data = df[col].dropna().values
        if len(data) == 0 or not np.isfinite(data).any():
            continue

        plt.figure(figsize=(6, 4))
        plt.hist(data, bins=30, color="skyblue", edgecolor="black")
        plt.title(f"Global Histogram of {col}")
        plt.xlabel(col)
        plt.ylabel("Frequency")
        plt.grid(True, linestyle="--", alpha=0.6)

        hist_path = os.path.join(output_folder, f"hist_global_{col}.png")
        plt.savefig(hist_path, dpi=150, bbox_inches="tight")
        plt.close()

# --------------------------
# Box, Violin, Scatter Plots
# --------------------------
def visualize_parameters(df, output_folder):
    param_units = {
        "outer_radius": "Outer Radius (µm)",
        "inner_radius": "Inner Radius (µm)",
        "thickness": "Thickness (µm)",
        "diameter": "Diameter (µm)",
        "outer_area": "Outer Area (µm²)",
        "inner_area": "Inner Area (µm²)",
        "area_ratio": "Area Ratio",
        "hole_ratio": "Hole Ratio",
        "ring_contrast": "Ring Contrast",
        "a_mean_center": "A-Mean Center (intensity)",
        "eccentricity": "Eccentricity",
        "g_ratio": "g-ratio",
        "num_inner_contours": "Number of Inner Contours"
    }

    df = df.copy()
    df = df.apply(pd.to_numeric, errors="ignore")

    # 1️⃣ Print all data
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    print("\n📋 Axon Data (All rows):")
    print(df)

    # 2️⃣ Individual plots
    for param, label in param_units.items():
        if param not in df:
            continue
        data = df[param].dropna()
        if len(data) == 0:
            continue

        # Boxplot
        plt.figure(figsize=(6, 4))
        sns.boxplot(x=data, color="skyblue")
        plt.title(f"Boxplot of {label}")
        plt.xlabel(label)
        plt.savefig(os.path.join(output_folder, f"box_{param}.png"), dpi=150, bbox_inches="tight")
        plt.close()

        # Violin plot
        plt.figure(figsize=(6, 4))
        sns.violinplot(x=data, inner="box", color="lightgreen")
        plt.title(f"Violin Plot of {label}")
        plt.xlabel(label)
        plt.savefig(os.path.join(output_folder, f"violin_{param}.png"), dpi=150, bbox_inches="tight")
        plt.close()

        # Histogram
        plt.figure(figsize=(6, 4))
        sns.histplot(data, kde=True, bins=30, color="blue")
        plt.title(f"Histogram of {label}")
        plt.xlabel(label)
        plt.ylabel("Count")
        plt.savefig(os.path.join(output_folder, f"hist_{param}.png"), dpi=150, bbox_inches="tight")
        plt.close()

    # Comparative boxplot
    df_melted = df[list(param_units.keys())].melt(var_name="Parameter", value_name="Value")
    plt.figure(figsize=(12, 6))
    sns.boxplot(x="Parameter", y="Value", data=df_melted)
    plt.xticks(rotation=45, ha="right")
    plt.title("Comparative Boxplots of Axon Parameters")
    plt.ylabel("Value (with units depending on parameter)")
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, "comparative_boxplots.png"), dpi=150)
    plt.close()

    # Scatter plot matrix
    clean_df = df[list(param_units.keys())].dropna()
    if not clean_df.empty:
        pairplot = sns.pairplot(clean_df, corner=True, plot_kws={"alpha": 0.5, "s": 20})
        pairplot.fig.suptitle("Scatter Plot Matrix of Axon Parameters", y=1.02)
        pairplot.savefig(os.path.join(output_folder, "scatter_matrix.png"), dpi=150, bbox_inches="tight")
        plt.close("all")

    print(f"\n✅ Plots saved in '{output_folder}'")

# --------------------------
# Summary counts
# --------------------------
def save_summary(df, output_folder):
    mature_count = (df["axon_type"] == "mature").sum()
    regrowth_clusters = (df["axon_type"] == "regrowth_cluster").sum()
    total_regrowth_axons = df.loc[df["axon_type"] == "regrowth_cluster", "num_inner_contours"].sum()

    summary_text = (
        f"Number of mature axons: {mature_count}\n"
        f"Number of regrowth clusters: {regrowth_clusters}\n"
        f"Number of total regrowth axons: {total_regrowth_axons}\n"
    )

    print("\n📊 Summary:\n" + summary_text)

    with open(os.path.join(output_folder, "summary.txt"), "w") as f:
        f.write(summary_text)

# --------------------------
# Run Pipeline
# --------------------------
axon_data, output_image = process_image(image_path, patch_size=1024)
df = save_to_csv(axon_data, output_folder)
plot_global_histograms(df, output_folder)
visualize_parameters(df, output_folder)
save_summary(df, output_folder)

print(f"\n✅ All CSV + plots + summary saved in '{output_folder}'")

C:\Users\DELL\AppData\Local\Temp\ipykernel_27148\2253773320.py:187: RuntimeWarning: invalid value encountered in sqrt
  eccentricity = np.sqrt(1 - (ma / MA) ** 2)


Axon 1 (mature):
 Center = (2514.93, 2207.17)
 Outer Radius = 3.6420 µm
 Inner Radius = 0.6269 µm
 Thickness = 3.0151 µm
 Diameter = 7.2840 µm
 Outer Area = 9.0723 µm²
 Inner Area = 0.6566 µm²
 G-Ratio = 0.1721
 Inner Contours = 1

Axon 2 (regrowth_cluster):
 Center = (4027.00, 3008.50)
 Outer Radius = 3.7769 µm
 Inner Radius = 1.7654 µm
 Thickness = 2.0115 µm
 Diameter = 7.5538 µm
 Outer Area = 18.0059 µm²
 Inner Area = 4.8460 µm²
 G-Ratio = 0.4674
 Inner Contours = 2

Axon 3 (mature):
 Center = (4070.00, 2927.50)
 Outer Radius = 2.8177 µm
 Inner Radius = 1.9825 µm
 Thickness = 0.8352 µm
 Diameter = 5.6354 µm
 Outer Area = 17.0533 µm²
 Inner Area = 4.8367 µm²
 G-Ratio = 0.7036
 Inner Contours = 1

Axon 4 (mature):
 Center = (4958.50, 3022.50)
 Outer Radius = 2.1611 µm
 Inner Radius = 0.9665 µm
 Thickness = 1.1946 µm
 Diameter = 4.3222 µm
 Outer Area = 7.1764 µm²
 Inner Area = 1.3410 µm²
 G-Ratio = 0.4472
 Inner Contours = 1

Axon 5 (mature):
 Center = (5062.19, 2990.86)
 Outer Radius 

C:\Users\DELL\AppData\Local\Temp\ipykernel_27148\2253773320.py:311: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors="ignore")



📋 Axon Data (All rows):
     axon_id         axon_type  num_inner_contours      center_x  \
0          1            mature                   1   2514.931915   
1          2  regrowth_cluster                   2   4027.000000   
2          3            mature                   1   4070.000000   
3          4            mature                   1   4958.500000   
4          5            mature                   1   5062.192322   
5          6            mature                   1   5072.290710   
6          7            mature                   1   4887.490723   
7          8            mature                   1   4538.787384   
8          9            mature                   1   4779.667969   
9         10            mature                   1   4724.725830   
10        11            mature                   1   4889.500000   
11        12  regrowth_cluster                   2   4273.962433   
12        13            mature                   1   4372.007874   
13        14           

In [9]:
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

# --------------------------
# Input and output paths
# --------------------------
image_path = r"C:\Users\sanje\Downloads\low density clip 2.png"
output_folder = r"C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_g_ar"
os.makedirs(output_folder, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245
MIN_CIRCULARITY = 0.10
MIN_CONTOUR_AREA = 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.3  # adjust based on dataset

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)
    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset, axon_data, object_counter, output_image):
    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    # Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)

    # Thresholding
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )
    thresh = cv2.bitwise_or(otsu, adaptive)

    # Clean
    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    contours, hierarchy = cv2.findContours(cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):
        if hierarchy[i][3] != -1:  # only outer contours
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # Inner children
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)
        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) == 1 else "regrowth_cluster"
        largest_inner = max(inner_children, key=cv2.contourArea)

        # Outer
        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset
        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor**2)

        # Inner
        (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(largest_inner)
        inner_shifted = largest_inner + np.array([x_offset, y_offset])
        cv2.drawContours(output_image, [contour_shifted], -1, (255, 0, 0), 1)
        cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
        inner_area_px = cv2.contourArea(largest_inner)
        inner_area_um2 = inner_area_px * (scale_factor**2)

        # Metrics
        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "diameter": (2 * outer_radius) * scale_factor,
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "g_ratio": g_ratio
        })

        # Numbering
        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

        object_counter += 1

    return object_counter

# --------------------------
# Whole image tiling
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError("Could not read image")

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]
    output_image = image.copy()
    axon_data = []
    object_counter = 1

    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]
            object_counter = process_patch(
                patch, mask_patch, x, y, axon_data, object_counter, output_image
            )

    filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, f"numbered_{filename}")
    cv2.imwrite(output_path, output_image)

    print(f"\nProcessed: {filename}")
    print(f"Axons detected: {len(axon_data)}\n")
    return axon_data, output_image

# --------------------------
# Save data to CSV
# --------------------------
def save_to_csv(axon_data, output_folder, filename="axon_measurements.csv"):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, filename)
    df.to_csv(csv_path, index=False)
    print(f"✅ Axon data saved to: {csv_path}")
    return df

# --------------------------
# G-ratio Analysis
# --------------------------
def analyze_g_ratio(df, output_folder, threshold_range=(0.6, 0.8)):
    # Total axons
    total_axons = len(df)
    print(f"\n📊 Total axons detected: {total_axons}")

    # G-ratio values
    g_ratios = df["g_ratio"].dropna().values
    mean_g = np.mean(g_ratios) if len(g_ratios) > 0 else None
    median_g = np.median(g_ratios) if len(g_ratios) > 0 else None
    print(f"✅ Computed g-ratios for {len(g_ratios)} axons")

    # Explain calculation
    print("\nℹ️ G-ratio is calculated as:")
    print("   g_ratio = inner_radius / outer_radius")
    print("   (both radii in micrometers after scaling)\n")

    # --- Plot 1: With expected range + mean line ---
    plt.figure(figsize=(6, 4))
    plt.hist(g_ratios, bins=30, color="lightblue", edgecolor="black", alpha=0.7)

    plt.axvspan(threshold_range[0], threshold_range[1], color="red", alpha=0.2,
                label=f"Expected range {threshold_range[0]}–{threshold_range[1]}")

    if mean_g is not None:
        plt.axvline(mean_g, color="blue", linestyle="--", linewidth=2,
                    label=f"Mean g-ratio = {mean_g:.3f}")

    plt.title("G-ratio Distribution (with expected range)")
    plt.xlabel("g-ratio")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.6)

    out_path1 = os.path.join(output_folder, "g_ratio_distribution_with_range.png")
    plt.savefig(out_path1, dpi=150, bbox_inches="tight")
    plt.close()

    # --- Plot 2: Only mean line ---
    plt.figure(figsize=(6, 4))
    plt.hist(g_ratios, bins=30, color="lightblue", edgecolor="black", alpha=0.7)

    if mean_g is not None:
        plt.axvline(mean_g, color="blue", linestyle="--", linewidth=2,
                    label=f"Mean g-ratio = {mean_g:.3f}")

    plt.title("G-ratio Distribution (mean only)")
    plt.xlabel("g-ratio")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.6)

    out_path2 = os.path.join(output_folder, "g_ratio_distribution_mean_only.png")
    plt.savefig(out_path2, dpi=150, bbox_inches="tight")
    plt.close()

    # --- Plot 3: Mean + Median lines ---
    plt.figure(figsize=(6, 4))
    plt.hist(g_ratios, bins=30, color="lightblue", edgecolor="black", alpha=0.7)

    if mean_g is not None:
        plt.axvline(mean_g, color="blue", linestyle="--", linewidth=2,
                    label=f"Mean g-ratio = {mean_g:.3f}")
    if median_g is not None:
        plt.axvline(median_g, color="green", linestyle="-.", linewidth=2,
                    label=f"Median g-ratio = {median_g:.3f}")

    plt.title("G-ratio Distribution (mean vs median)")
    plt.xlabel("g-ratio")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.6)

    out_path3 = os.path.join(output_folder, "g_ratio_distribution_mean_median.png")
    plt.savefig(out_path3, dpi=150, bbox_inches="tight")
    plt.close()

    print(f"📈 Saved plots:\n - {out_path1}\n - {out_path2}\n - {out_path3}")

    return total_axons, g_ratios, mean_g, median_g



# --------------------------
# Complete plots for mature vs regenerated axons
# --------------------------
def analyze_area_and_g_ratio_complete(df, output_folder):
    df['area_ratio'] = df['inner_area'] / (df['outer_area'] + 1e-6)

    axon_types = ['mature', 'regrowth_cluster']
    metrics = ['area_ratio', 'g_ratio']

    colors = {
        'mature': {'area_ratio': 'skyblue', 'g_ratio': 'lightgreen'},
        'regrowth_cluster': {'area_ratio': 'salmon', 'g_ratio': 'lightcoral'}
    }

    for ax_type in axon_types:
        subset = df[df['axon_type'] == ax_type]
        for metric in metrics:
            data = subset[metric].dropna()
            if len(data) == 0:
                continue

            # Histogram
            plt.figure(figsize=(6,4))
            plt.hist(data, bins=30, color=colors[ax_type][metric], edgecolor='black', alpha=0.7)
            plt.title(f"{metric.replace('_',' ').title()} Histogram ({ax_type.title()} Axons)")
            plt.xlabel(metric.replace('_',' ').title())
            plt.ylabel("Frequency")
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.savefig(os.path.join(output_folder, f"hist_{metric}_{ax_type}.png"), dpi=150, bbox_inches='tight')
            plt.close()

            # Boxplot
            plt.figure(figsize=(4,6))
            plt.boxplot(data, vert=True, patch_artist=True,
                        boxprops=dict(facecolor=colors[ax_type][metric], color='black'))
            plt.title(f"{metric.replace('_',' ').title()} Boxplot ({ax_type.title()} Axons)")
            plt.ylabel(metric.replace('_',' ').title())
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.savefig(os.path.join(output_folder, f"box_{metric}_{ax_type}.png"), dpi=150, bbox_inches='tight')
            plt.close()

    print("📊 Complete plots saved for area_ratio and g_ratio by axon type (mature & regrowth).")

# --------------------------
# Run complete analysis plots
# --------------------------


# --------------------------
# Summary
# --------------------------
def save_summary(df, output_folder, threshold_range=(0.6, 0.8)):
    summary_path = os.path.join(output_folder, "summary.txt")

    total_axons = len(df)
    g_ratios = df["g_ratio"].dropna().values

    mean_g = np.mean(g_ratios) if len(g_ratios) > 0 else None
    median_g = np.median(g_ratios) if len(g_ratios) > 0 else None

    # Percentage within threshold range
    if len(g_ratios) > 0:
        within_range = np.sum((g_ratios >= threshold_range[0]) & (g_ratios <= threshold_range[1]))
        percent_in_range = (within_range / len(g_ratios)) * 100
    else:
        percent_in_range = 0.0

    with open(summary_path, "w") as f:
        f.write("Axon Analysis Summary\n")
        f.write("====================\n\n")
        f.write(f"Total axons detected: {total_axons}\n")
        if mean_g is not None:
            f.write(f"Mean g-ratio: {mean_g:.4f}\n")
        if median_g is not None:
            f.write(f"Median g-ratio: {median_g:.4f}\n")
        f.write(f"Axons within expected range {threshold_range[0]}–{threshold_range[1]}: {percent_in_range:.2f}%\n")

    print(f"📝 Summary saved at: {summary_path}")


# --------------------------
# Run Pipeline
# --------------------------
axon_data, output_image = process_image(image_path, patch_size=1024)
df = save_to_csv(axon_data, output_folder)
total_axons, g_ratios, mean_g, median_g = analyze_g_ratio(df, output_folder)
analyze_area_and_g_ratio_complete(df, output_folder)
save_summary(df, output_folder)

print(f"\n✅ Done: CSV, g-ratio analysis, and summary saved in '{output_folder}'")
#with g-ratio, and diff btw regrowth and mature for area-ratio and g-ratio


Processed: low density clip 2.png
Axons detected: 125

✅ Axon data saved to: C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_g_ar\axon_measurements.csv

📊 Total axons detected: 125
✅ Computed g-ratios for 125 axons

ℹ️ G-ratio is calculated as:
   g_ratio = inner_radius / outer_radius
   (both radii in micrometers after scaling)

📈 Saved plots:
 - C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_g_ar\g_ratio_distribution_with_range.png
 - C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_g_ar\g_ratio_distribution_mean_only.png
 - C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_g_ar\g_ratio_distribution_mean_median.png
📊 Complete plots saved for area_ratio and g_ratio by axon type (mature & regrowth).
📝 Summary saved at: C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_g_ar\summary.txt

✅ Done: CSV, g-ratio analysis, and summary saved in 'C:\Users\sanje\Dow

In [ ]:
"C:\Users\sanje\Downloads\high density clip.png"
"C:\Users\sanje\Downloads\low density clip 2.png"

In [10]:

import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

# --------------------------
# Input and output paths
# --------------------------
image_path = r"C:\Users\sanje\Downloads\low density clip 2.png"
output_folder = r"C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_gvsdia2"
os.makedirs(output_folder, exist_ok=True)

# Scale factor (µm/px)
scale_factor = 0.136

# --------------------------
# Tunables
# --------------------------
TISSUE_OVERLAP_MIN = 0.50
MAX_MEAN_INTENSITY = 245
MIN_CIRCULARITY = 0.10
MIN_CONTOUR_AREA = 20
MAX_CONTOUR_AREA = 2000000
MIN_SOLIDITY = 0.3  # adjust based on dataset

# --------------------------
# Helpers
# --------------------------
def darken_and_sharpen(image):
    darkened = cv2.convertScaleAbs(image, alpha=1.5, beta=-20)
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    return cv2.filter2D(darkened, -1, sharpen_kernel)

def build_tissue_mask(full_image):
    hsv = cv2.cvtColor(full_image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)
    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)
    mask = cv2.bitwise_and(sat_mask, not_too_bright)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    return mask

def contour_overlap_ratio(contour, mask):
    c_mask = np.zeros(mask.shape, dtype=np.uint8)
    cv2.drawContours(c_mask, [contour], -1, 255, -1)
    inter = cv2.bitwise_and(c_mask, mask)
    area_c = cv2.countNonZero(c_mask)
    area_i = cv2.countNonZero(inter)
    return area_i / float(area_c) if area_c > 0 else 0.0

def circularity(contour):
    a = cv2.contourArea(contour)
    p = cv2.arcLength(contour, True)
    return 4 * np.pi * a / (p * p + 1e-6) if p > 0 else 0

# --------------------------
# Main patch processing
# --------------------------
def process_patch(patch, mask_patch, x_offset, y_offset, axon_data, object_counter, output_image):
    patch = darken_and_sharpen(patch)
    gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    # Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)

    # Thresholding
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    _, otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    adaptive = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 35, 2
    )
    thresh = cv2.bitwise_or(otsu, adaptive)

    # Clean
    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    contours, hierarchy = cv2.findContours(cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if hierarchy is None:
        return object_counter
    hierarchy = hierarchy[0]

    for i, contour in enumerate(contours):
        if hierarchy[i][3] != -1:  # only outer contours
            continue

        area = cv2.contourArea(contour)
        if area < MIN_CONTOUR_AREA or area > MAX_CONTOUR_AREA:
            continue

        overlap = contour_overlap_ratio(contour, mask_patch)
        if overlap < TISSUE_OVERLAP_MIN:
            continue

        hull = cv2.convexHull(contour)
        solidity = area / (cv2.contourArea(hull) + 1e-6)
        circ = circularity(contour)
        if solidity < MIN_SOLIDITY or circ < MIN_CIRCULARITY:
            continue

        # Inner children
        inner_children = []
        for j, child in enumerate(contours):
            if hierarchy[j][3] == i and cv2.contourArea(child) > 30:
                inner_children.append(child)
        if len(inner_children) == 0:
            continue

        axon_type = "mature" if len(inner_children) == 1 else "regrowth_cluster"
        largest_inner = max(inner_children, key=cv2.contourArea)

        # Outer
        (x_outer, y_outer), outer_radius = cv2.minEnclosingCircle(contour)
        contour_shifted = contour + np.array([x_offset, y_offset])
        x_outer += x_offset
        y_outer += y_offset
        outer_area_px = cv2.contourArea(contour)
        outer_area_um2 = outer_area_px * (scale_factor**2)

        # Inner
        (x_inner, y_inner), inner_radius = cv2.minEnclosingCircle(largest_inner)
        inner_shifted = largest_inner + np.array([x_offset, y_offset])
        cv2.drawContours(output_image, [contour_shifted], -1, (255, 0, 0), 1)
        cv2.drawContours(output_image, [inner_shifted], -1, (0, 255, 0), 1)
        inner_area_px = cv2.contourArea(largest_inner)
        inner_area_um2 = inner_area_px * (scale_factor**2)

        # Metrics
        thickness = (outer_radius - inner_radius) * scale_factor
        g_ratio = inner_radius / (outer_radius + 1e-6)

        axon_data.append({
            "axon_id": object_counter,
            "axon_type": axon_type,
            "outer_radius": outer_radius * scale_factor,
            "inner_radius": inner_radius * scale_factor,
            "thickness": thickness,
            "inner_diameter": (2 * inner_radius) * scale_factor,
            "axon_diameter": (2 * inner_radius) * scale_factor,  # <-- INNER diameter
            "outer_area": outer_area_um2,
            "inner_area": inner_area_um2,
            "g_ratio": g_ratio
        })

        # Numbering
        cv2.putText(output_image, str(object_counter),
                    (int(x_outer), int(y_outer)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

        object_counter += 1

    return object_counter

# --------------------------
# Whole image tiling
# --------------------------
def process_image(image_path, patch_size=1024):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError("Could not read image")

    tissue_mask_full = build_tissue_mask(image)
    h, w = image.shape[:2]
    output_image = image.copy()
    axon_data = []
    object_counter = 1

    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            mask_patch = tissue_mask_full[y:y+patch_size, x:x+patch_size]
            object_counter = process_patch(
                patch, mask_patch, x, y, axon_data, object_counter, output_image
            )

    filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, f"numbered_{filename}")
    cv2.imwrite(output_path, output_image)

    print(f"\nProcessed: {filename}")
    print(f"Axons detected: {len(axon_data)}\n")
    return axon_data, output_image

# --------------------------
# Save data to CSV
# --------------------------
def save_to_csv(axon_data, output_folder, filename="axon_measurements.csv"):
    df = pd.DataFrame(axon_data)
    csv_path = os.path.join(output_folder, filename)
    df.to_csv(csv_path, index=False)
    print(f"✅ Axon data saved to: {csv_path}")
    return df

# --------------------------
# Scatter: Diameter vs g-ratio
# --------------------------
def plot_diameter_vs_gratio(df, output_folder):
    plt.figure(figsize=(7, 5))

    # Scatter for mature axons
    mature = df[df["axon_type"] == "mature"]
    plt.scatter(
        mature["axon_diameter"], mature["g_ratio"],
        c="blue", label="Mature", alpha=0.7, edgecolor="k", s=50
    )

    # Scatter for regenerating axons
    regrowth = df[df["axon_type"] == "regrowth_cluster"]
    plt.scatter(
        regrowth["axon_diameter"], regrowth["g_ratio"],
        c="red", label="Regrowth", alpha=0.7, edgecolor="k", s=50
    )

    plt.xlabel("Inner Axon Diameter (µm)")
    plt.ylabel("G-ratio")
    plt.title("Inner-based Axon Diameter vs G-ratio")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.5)

    out_path = os.path.join(output_folder, "axon_diameter_vs_g_ratio.png")
    plt.savefig(out_path, dpi=150, bbox_inches="tight")
    plt.close()

    print(f"📈 Scatter plot saved: {out_path}")


# --------------------------
# Summary statistics
# --------------------------
# --------------------------
# Summary statistics (Diameter + Thickness)
# --------------------------
def summarize_statistics(df, output_folder):
    stats = {}

    for axon_type, subset in {
        "Regrowth Axons": df[df["axon_type"] == "regrowth_cluster"],
        "Mature Axons": df[df["axon_type"] == "mature"],
        "All Axons": df
    }.items():
        if len(subset) == 0:
            continue
        stats[axon_type] = {
            "Diameter_min": subset["axon_diameter"].min(),
            "Diameter_max": subset["axon_diameter"].max(),
            "Diameter_mean": subset["axon_diameter"].mean(),
            "Diameter_median": subset["axon_diameter"].median(),
            "Thickness_min": subset["thickness"].min(),
            "Thickness_max": subset["thickness"].max(),
            "Thickness_mean": subset["thickness"].mean(),
            "Thickness_median": subset["thickness"].median()
        }

    # Convert to dataframe for pretty export
    stats_df = pd.DataFrame(stats).T.round(3)  # round to 3 decimals
    csv_path = os.path.join(output_folder, "axon_summary_statistics.csv")
    stats_df.to_csv(csv_path)
    print(f"📊 Summary statistics saved: {csv_path}")
    print(stats_df)

    return stats_df

# --------------------------
# Scatter: Diameter vs Thickness
# --------------------------
def plot_diameter_vs_thickness(df, output_folder):
    plt.figure(figsize=(7, 5))

    # Mature axons → open circles (rings)
    mature = df[df["axon_type"] == "mature"]
    plt.scatter(
        mature["axon_diameter"], mature["thickness"],
        facecolors='none', edgecolors="blue", label="Mature", s=70, linewidth=1.2
    )

    # Regrowth axons → solid dots
    regrowth = df[df["axon_type"] == "regrowth_cluster"]
    plt.scatter(
        regrowth["axon_diameter"], regrowth["thickness"],
        c="red", label="Regrowth", s=60, alpha=0.8
    )

    plt.xlabel("Inner Axon Diameter (µm)")
    plt.ylabel("Myelin Thickness (µm)")
    plt.title("Inner Axon Diameter vs Myelin Thickness")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.5)

    out_path = os.path.join(output_folder, "axon_diameter_vs_thickness.png")
    plt.savefig(out_path, dpi=150, bbox_inches="tight")
    plt.close()

    print(f"📈 Scatter plot saved: {out_path}")

# --------------------------
# Run Pipeline
# --------------------------
axon_data, output_image = process_image(image_path, patch_size=1024)
df = save_to_csv(axon_data, output_folder)
plot_diameter_vs_gratio(df, output_folder)
plot_diameter_vs_thickness(df, output_folder) 
summarize_statistics(df, output_folder)
print(f"\n✅ Done: CSV, scatter plot saved in '{output_folder}'")
#gratio vs inner diameter, inner diamter vs thickness and has the mean median min max of diamter and thickness


Processed: low density clip 2.png
Axons detected: 125

✅ Axon data saved to: C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_gvsdia2\axon_measurements.csv
📈 Scatter plot saved: C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_gvsdia2\axon_diameter_vs_g_ratio.png
📈 Scatter plot saved: C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_gvsdia2\axon_diameter_vs_thickness.png
📊 Summary statistics saved: C:\Users\sanje\Downloads\lowdensity_qwerty5_4_statistics_regrowthandmature_gvsdia2\axon_summary_statistics.csv
                Diameter_min  Diameter_max  Diameter_mean  Diameter_median  \
Regrowth Axons         1.566        10.888          4.227            3.662   
Mature Axons           1.122         7.849          2.989            2.880   
All Axons              1.122        10.888          3.256            2.907   

                Thickness_min  Thickness_max  Thickness_mean  Thickness_median  
Regrowth Axons